In [2]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#pd.reset_option('display.max_rows')

## Sacar links

In [4]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [7]:
anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

total_paginas

6531

In [9]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:5]
        
    for pagina_url in tqdm.tqdm(lista_paginas):
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("div", class_="contenido-anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')
            
        time.sleep(random.uniform(1,3))
    
        
    return lista_paginas, anuncios_coches

In [11]:
lista_paginas,anuncios_coches = links_coches()

100%|██████████| 6531/6531 [5:49:42<00:00,  3.21s/it]  


In [12]:
df_lista_coches = pd.DataFrame(anuncios_coches)
df_lista_coches.to_csv('../bin/anuncios_coches.csv', index = False)

In [13]:
len(anuncios_coches)

45436

## Sacar coches

In [5]:
df_lista_coches = pd.read_csv('../bin/anuncios_coches.csv')

In [9]:
lista_jorge = df_lista_coches.iloc[:15147]
lista_miguel = df_lista_coches.iloc[15147:30294]
lista_carlos = df_lista_coches.iloc[30294:]

In [11]:
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
session = requests.Session()

In [13]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [17]:
coches_df = dict()

for coche in tqdm.tqdm(lista_jorge.iloc[:, 0]):
    print(coche)
    
    try: 
        url_coche = coche
        response = session.get(url_coche, headers=headers)
        
    except: 
        continue
        
    #Verificamos que la respuesta es exitosa
    if response.status_code != 200:
        print(f"Error al acceder a {url_coche}, Código de estado: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    info_coche = {}
    
    try:
        if 'Vehículo vendido' in soup.find('div', class_= 'img-ficha').find('p').text.replace('X',''):
            continue
            
    except: 
        
        pass
            
    try: 
        info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
        
    except: 
        info_coche['provincia'] = 'no disponible'
        
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
        
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        try: 
            detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
            for detalle in detalles_carroceria:
                detalle = detalle.text.replace('Medidas','').strip()

                if 'Largo' in detalle: 
                    info_coche['largo'] = detalle
                elif 'Ancho' in detalle: 
                    info_coche['ancho'] = detalle
                elif 'Alto' in detalle: 
                    info_coche['alto'] = detalle
                #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
                elif 'Capacidad del maletero' in detalle: 
                    info_coche['capacidad_maletero'] = detalle
                elif 'Carrocería' in detalle: 
                    info_coche['carroceria'] = detalle
                elif 'Número de plazas' in detalle: 
                    info_coche['num_plazas'] = detalle
                elif 'Número de puertas' in detalle: 
                    info_coche['num_puertas'] = detalle
                elif 'Batalla' in detalle: 
                    info_coche['batalla'] = detalle
                #supongo que si hay dos pesos el segundo será la masa max. autorizada
                elif 'Peso' in detalle: 
                    info_coche['peso'] = detalle
                    
        except:   
            info_coche['largo'] = 'no disponible'
            info_coche['ancho'] = 'no disponible'
            info_coche['alto'] = 'no disponible'
            info_coche['capacidad_maletero'] = 'no disponible'
            info_coche['carroceria'] = 'no disponible'
            info_coche['num_plazas'] = 'no disponible'
            info_coche['num_puertas'] = 'no disponible'
            info_coche['batalla'] = 'no disponible'
            info_coche['peso'] = 'no disponible'
                
           
        try:
            detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
            for detalle in detalles_consumos:
                detalle = ' '.join(detalle.text.replace('\n','').split())

                if 'Consumo medio' in detalle: 
                    info_coche['consumo_medio'] = detalle
                elif 'Consumo en carretera' in detalle: 
                    info_coche['consumo_carretera'] = detalle
                elif 'Consumo urbano' in detalle: 
                    info_coche['consumo_urbano'] = detalle
                elif 'CO2' in detalle: 
                    info_coche['CO2'] = detalle
                elif 'Depósito' in detalle: 
                    info_coche['deposito'] = detalle
                    
        except: 
            info_coche['consumo_medio'] = 'no disponible'
            info_coche['consumo_carretera'] = 'no disponible'
            info_coche['consumo_urbano'] = 'no disponible'
            info_coche['CO2'] = 'no disponible'
            info_coche['deposito'] = 'no disponible'        
        
        try:
            detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
            for detalle in detalles_motor:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Combustible' in detalle: 
                    info_coche['combustible'] = detalle
                elif 'Nº de cilindros' in detalle: 
                    info_coche['cilindros'] = detalle
                elif 'Cilindrada' in detalle: 
                    info_coche['cilindrada'] = detalle
                elif 'Sobrealimentación' in detalle: 
                    info_coche['sobrealimentacion'] = detalle
                    
        except: 
            info_coche['combustible'] = 'no disponible'
            info_coche['cilindros'] = 'no disponible'
            info_coche['cilindrada'] = 'no disponible'
            info_coche['sobrealimentacion'] = 'no disponible'

        try:
            detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
            for detalle in detalles_transmision:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Tipo de transmisión' in detalle: 
                    info_coche['tipo_cambio'] = detalle
                elif 'Tracción' in detalle: 
                    info_coche['traccion'] = detalle
                elif 'Número de marchas' in detalle: 
                    info_coche['num_marchas'] = detalle
                    
        except: 
            info_coche['tipo_cambio'] = 'no disponible'
            info_coche['traccion'] = 'no disponible'
            info_coche['num_marchas'] = 'no disponible'

        try:
            detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
            for detalle in detalles_prestaciones:
                detalle = detalle.text

                if 'Potencia (KW)' in detalle: 
                    info_coche['potencia_kw'] = detalle
                elif 'Potencia (CV)' in detalle: 
                    info_coche['potencia_cv'] = detalle
                elif 'Par' in detalle: 
                    info_coche['par'] = detalle
                elif 'Velocidad' in detalle: 
                    info_coche['velocidad_max'] = detalle
                elif 'Aceleración' in detalle: 
                    info_coche['aceleracion'] = detalle
                    
        except: 
            info_coche['potencia_kw'] = 'no disponible'
            info_coche['potencia_cv'] = 'no disponible'
            info_coche['par'] = 'no disponible'
            info_coche['velocidad_max'] = 'no disponible'
            info_coche['aceleracion'] = 'no disponible'
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(3, 7))


  0%|          | 0/15147 [00:00<?, ?it/s]

https://www.autocasion.com/coches-segunda-mano/bmw-serie-3-ocasion/318d-7-ref14633654


  0%|          | 1/15147 [00:05<21:50:19,  5.19s/it]

https://www.autocasion.com/coches-segunda-mano/fiat-500-ocasion/500-1-0-hybrid-dolcevita-52kw-ref14498767


  0%|          | 2/15147 [00:12<26:07:54,  6.21s/it]

https://www.autocasion.com/coches-segunda-mano/nissan-qashqai-ocasion/qashqai-1-5dci-n-connecta-4x2-ref14512790


  0%|          | 2/15147 [00:16<35:22:01,  8.41s/it]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame.from_dict(coches_df).T

In [ ]:
df

In [ ]:
df.to_csv('../bin/data.csv', index=False)

## Extracción listado marcas

In [44]:
url = 'https://www.autocasion.com/marcas'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [66]:
marcas = soup.find('ul', class_= 'marcas').find_all('span')


listado_marcas = {}
for index,detalle in enumerate (marcas, start = 1):
    detalle = detalle.text.strip()
    
    listado_marcas[index] = detalle
    

In [68]:
listado_marcas

{1: 'Abarth',
 2: 'Aiways',
 3: 'Alfa romeo',
 4: 'Alpine',
 5: 'Aston martin',
 6: 'Audi',
 7: 'Bentley',
 8: 'Bmw',
 9: 'Byd',
 10: 'Citroen',
 11: 'Cupra',
 12: 'Dacia',
 13: 'Dfsk',
 14: 'Dongfeng',
 15: 'Dr',
 16: 'Ds',
 17: 'Evo',
 18: 'Ferrari',
 19: 'Fiat',
 20: 'Ford',
 21: 'Honda',
 22: 'Hyundai',
 23: 'Ineos',
 24: 'Invicta',
 25: 'Isuzu',
 26: 'Jaecoo',
 27: 'Jaguar',
 28: 'Jeep',
 29: 'Jiayuan',
 30: 'Kgm / ssangyong',
 31: 'Kia',
 32: 'Lamborghini',
 33: 'Lancia',
 34: 'Land-rover',
 35: 'Leapmotor',
 36: 'Levc',
 37: 'Lexus',
 38: 'Livan',
 39: 'Lotus',
 40: 'Lynk & co',
 41: 'Maserati',
 42: 'Maxus',
 43: 'Mazda',
 44: 'Mercedes-benz',
 45: 'Mg',
 46: 'Mhero',
 47: 'Mini',
 48: 'Mitsubishi',
 49: 'Morgan',
 50: 'Nissan',
 51: 'Omoda',
 52: 'Opel',
 53: 'Peugeot',
 54: 'Polestar',
 55: 'Porsche',
 56: 'Renault',
 57: 'Seat',
 58: 'Seres',
 59: 'Skoda',
 60: 'Smart',
 61: 'Subaru',
 62: 'Suzuki',
 63: 'Swm',
 64: 'Tesla',
 65: 'Toyota',
 66: 'Volkswagen',
 67: 'Volvo',
 6

In [70]:
df_marcas = pd.DataFrame(list(listado_marcas.items()), columns=['id_marca', 'nombre_marca'])

In [72]:
df_marcas.to_csv('../bin/listado_marcas.csv', index=False)

## Pruebas

In [21]:
url = 'https://www.autocasion.com/coches-ocasion?direction=desc&sort=updated_at&page=1'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [23]:
soup.find_all("article", class_="anuncio destacado")

[]

In [25]:
soup.find_all("article", class_="anuncio").find('a')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [123]:
soup.find_all("div", class_="contenido-anuncio")

[<div class="contenido-anuncio">
 <a href="/coches-segunda-mano/peugeot-508-ocasion/508-sw-1-6bluehdi-active-eat6-120-ref12543549"><h2 itemprop="name">  PEUGEOT 508 SW 1.6BlueHDI Active EAT6 120  </h2></a>
 <p class="precio">
 <span>Al contado</span>
                     11.500 €
                                     </p>
 <ul>
 <li>2017</li>
 <li>Diésel</li>
 <li>189.000 km</li>
 <li>
                         Madrid
                     </li>
 </ul>
 </div>,
 <div class="contenido-anuncio">
 <a href="/coches-segunda-mano/mercedes-benz-clase-glc-ocasion/glc-250-4matic-aut-ref14656275"><h2 itemprop="name">  MERCEDES-BENZ Clase GLC 250 4Matic Aut.  </h2></a>
 <p class="precio">
 <span>Al contado</span>
                     35.900 €
                                     </p>
 <ul>
 <li>2019</li>
 <li>Gasolina</li>
 <li>95.300 km</li>
 <li>
                         Madrid
                     </li>
 </ul>
 </div>,
 <div class="contenido-anuncio">
 <a href="/coches-km0/km-0/toyota-c-hr-km0/2-